In [2]:
!pip install qiskit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.8 MB/s 
     |████████████████████████████████| 19.2 MB 6.9 MB/s 
     |████████████████████████████████| 240 kB 64.0 MB/s 
     |████████████████████████████████| 112 kB 65.6 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 37.5 MB 49 kB/s 
     |████████████████████████████████| 943 kB 46.8 MB/s 
     |████████████████████████████████| 4.0 MB 55.7 MB/s 
     |████████████████████████████████| 1.9 MB 34.1 MB/s 
     |████████████████████████████████| 112 kB 49.1 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.0-py3-none-any.whl size=12188 sha256=d702034f43d2c1c913e91babf8c29dc192a78107a2b76c902f88ae8ecae05b83
  Stored in directory: /root/.cache/pip/wheels/63/d0/9a/257143

Importing all necessary modules

In [3]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.circuit.library.standard_gates import PhaseGate
from qiskit import Aer
import numpy as np

backend = Aer.get_backend('statevector_simulator')

Functions to define the multiplier

In [15]:
#convert decimals to binary
def binary(x,y):

  xb=bin(x)[2:]
  yb=bin(y)[2:]
  return xb, yb

#defining the multiplier now (function takes 2 decimals as input)
def multiplier(x,y):

    #to convert the decimal input into a binary representation using pre defined function "binary"
    xb, yb=binary(x,y)
    
    n=max(len(xb),len(yb))
    
    #to create registers
    xr = QuantumRegister(n, name="xr")
    yr = QuantumRegister(n, name="yr")
    z = QuantumRegister(2*n, name="z")
    c = ClassicalRegister(2*n,'c')

    #define quantum circuit
    qc = QuantumCircuit()
    qc.add_register(xr,yr,z,c)

    #encode the values
    q=0
    for i in "".join(reversed(xb)):
        if i=='1':
            qc.x(xr[q])
        q+=1
    
    q=0
    for i in "".join(reversed(yb)):
        if i=='1':
            qc.x(yr[q])
        q+=1

    #building the circuit 
    qc.append(QFT(2*n, do_swaps=False).to_gate(), z[:])

    for j in range(1, n + 1):
        for i in range(1, n + 1):
            for k in range(1, 2*n + 1):
                lam = (2 * np.pi) / (2 ** (i + j + k - 2 * n))
                qc.append(PhaseGate(lam).control(2),[xr[n - j], yr[n - i], z[k - 1]],)

    qc.append(QFT(2*n, do_swaps=False).inverse().to_gate(), z[:])

    for i in range(2*n):
        qc.measure(z[int(i)],c[i])
 

    print(qc)

    job = execute(qc, backend, shots=2500)
    result = job.result()
    counts = result.get_counts()
    ans=list(counts.keys())
    z=0
    q=0
    for i in "".join(reversed(ans[0])):
        z+=int(i)*(2**q)
        q+=1
    
    return z


Try the multiplier by inputting values

In [14]:
multiplier(7,8)

       ┌───┐                                                            »
xr_0: ─┤ X ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
xr_1: ─┤ X ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
xr_2: ─┤ X ├────────────────────────────────────────────────────────────»
       └───┘                                                            »
xr_3: ─────────■────────■────────■────────■───────■───────■───────■─────»
               │        │        │        │       │       │       │     »
yr_0: ─────────┼────────┼────────┼────────┼───────┼───────┼───────┼─────»
               │        │        │        │       │       │       │     »
yr_1: ─────────┼────────┼────────┼────────┼───────┼───────┼───────┼─────»
               │        │        │        │       │       │       │     »
yr_2: ─────────┼────────┼────────┼────

56

The result for 7*8 is 56, as shown by the multiplier.